# State Monad

## Outline

* Incentive for the State monad

* The State monad
  - Definition of the State monad

  - Helper functions

  - MonadState type class
  
  - Simple example
  
* State monad examples

In this lesson, we will learn about the State monad type and how you can use it. First we present the reason why introducing the State monad. Then we show the State monad definition and helper functions. We also look at the `MonadState` type class and in the end show some code examples that use the State monad.

## Incentive for State monad

We talked in the previous two lectures how you can use the Reader and Writer Monad if you have to read from a environment variable or write to it.

If you need to perform both operations you can use the State Monad. Let's first look at an example where we perform read and write operations without the State monad.

Our example program will asks the user for a password and perorm four checks on it:
- check1: Does the password have any duplicate letters (lower-upper case does not make a difference)

- check2: Is the password at least 10 characters long.

- check3: Are any symbols included in the password.

- check4: Is there at least one upper case and one lower case letter in the password.

In case one of the checks is not satisfied the program updates the password so that it passes the check.

If the update could influence a previous check the program performs the previous check again.

The program keeps also track of performed checks. Once the recursion limit of 20 checks is passed the checks stop. 

In the end the program notfies the user with the result of the check procedure.

In [ ]:
import Data.Char (toLower, toUpper)

type Password = String
type OperationsCount = Int
type CheckData = (OperationsCount, Password)

-- Removes duplicated characters
check1 :: CheckData -> CheckData
check1 (count,password) = if count > 20
    then (count + 1, "")
    else check2 (count + 1, filterPass password)
  where filterPass [] = []
        filterPass (x:xs) = x : filterPass (filter (filterFunc x) xs)
        filterFunc x = \char -> char `notElem` [toLower x,toUpper x]

-- Checks that the length of the password is not to small
check2 :: CheckData -> CheckData
check2 (count,password) = 
  if length password < 10
  then check1 (count + 1, extendedPassword)
  else check3 (count + 1, password)
  where shortage = 10 - length password
        passwordExtension = take shortage (drop count $ cycle allLetters)
        extendedPassword = password ++ passwordExtension

-- Checks if symbols are included
check3 :: CheckData -> CheckData
check3 (count,password) = if noSymbols
    then check4 (count + 1, extendedPassword)
    else check4 (count + 1, password)
  where noSymbols = all (`elem` allLetters) password
        passwordExtension = take 1 (drop count $ cycle "@!#$%&()?=")
        extendedPassword = password ++ passwordExtension

-- Checks there are at least 1 upper and 1 smaller letters
check4 :: CheckData -> CheckData
check4 (count,password)  
  | allLower = check1 (count + 1, upperExtendedPassword)
  | allUpper = check1 (count + 1, lowerExtendedPassword)
  | otherwise = (count + 1, password)
  where allLower = password == map toLower password
        allUpper = password == map toUpper password
        upperExtendedPassword = password ++ take 1 (drop count $ cycle upperLetters)
        lowerExtendedPassword = password ++ take 1 (drop count $ cycle lowerLetters)

lowerLetters :: String
lowerLetters = "abcdefghijklmnopqrstuvwxyz"

upperLetters :: String
upperLetters = "ABCDEFGHIJKLMNOPQRSTUVWXZY"

allLetters :: String
allLetters = lowerLetters ++ upperLetters

main :: IO ()
main = do
  putStrLn "Please input a password:"
  password <- getLine
  if null password
  then main
  else do
    let (count, checkedPassword) = check1 (0,password)
    if count > 20
    then putStrLn "Hit recursion limit of 20 password checks."
    else 
      if password == checkedPassword
      then putStrLn "Successfully performed all checks."
      else do
        putStrLn "Successfully performed all checks."
        putStrLn $ "Suggested improved passord: " ++ checkedPassword

main

Our program is fairly simple because it passes around only a tuple of two basic data types. 

Passing larger data around, reading and updating it can get more tedious when programs get bigger. 

This is where the **State monad** can help us out. 

## The State monad

### Definition of the State monad

The Haskell definition of the `State` type is defined in terms of the `StateT` monad transformer:
```haskell
type State s = StateT s Identity
```

It is contained in the **Control.Monad.Trans.State** module that comes with the **transformers** package. 

As said before we will learn about Monad transformers in lesson 25.

You can also define the State monad without using a monad transformer that works equally as the one above. 

Same as for the Reader and Writer monad we will chose this aproach. Let's first look at the definition of the **State** type:
```haskell
newtype State s a = State { runState :: s -> (a, s) }
```

We see that the `State` data constructor holds a function that can be accessed with the name `runState`.

It takes in a state and returns a tuple that contains a variable of type `a` and another state variable.

Same as for Reader and Writer monad we can now create a Monad instance for `State s` and not just `State`.

This means the type of our state will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: State s a        ->
         (a -> State s b) ->
         State s b
```

The monad instance for `State s` would be defined as:
```haskell
instance Monad (State s) where
    return a = State $ \st -> (a, st)
    stMonad >>= f = State $ \st1 -> 
        let (a, st2) = runState stMonad st1
        in runState (f a) st2
```

Also here we need to define the Functor and Applicative instances.
```haskell
instance Functor (State s) where
fmap f stFunctor = State $ \st1 ->
    let (a, st2) = runState stFunctor st1
    in (f a, st2)

instance Applicative (State s) where
pure a = State $ \st -> (a, st)
f <*> g = State $ \st1 ->
    let (h, st2) = runState f st1
        (a, st3) = runState g st2
    in (h a, st3)
```

### Helper functions

The **Control.Monad.Trans.State** module also defines some helper functions for the State moand. 

You have the option to import this module as a lazy or a strict module by appending **.Lazy** or **.Strict** to the module name.

Commonly used helper functions are `get` and `put` that are used inside a State monad. 
```haskell
get :: State s s
get = State $ \st -> (st, st)

put :: s -> State s ()
put st = State $ \_ -> ((), st)
```

The `get` function gets the state variable of the State monad. And the `put` function sets the state variable of the State monad. 

They are both used inside the State monad. To get the state variable you need to use the `<-` oprator that takes variables out of the monadic context.

A simple workflow in a State monad can contain the code pattern below:
```haskell
st <- get
let st' = f st
put st' 
```

Another helper function is the `state` function which performs the inverse operation of the `runState` function.

It takes in a function of type `s -> (a, s)` and produces a State monad.
```haskell
state :: (s -> (a, s)) -> State s a
state f = do
    st <- get
    let ~(a, st') = f st
    put st'
    return a
```

You can run it inside a state monad to update the state variable by applying the input function to it. 

Then there are the `evalState` and `execState` helper functions.
```haskell
evalState :: State s a -> s -> a
evalState stMonad st =
    fst $ runState stMonad st

execState :: State s a -> s -> s
execState stMonad st =
    snd $ runState stMonad st
```

The `evalState` function returns the result variable of the State monad. The `execState` function returns the state variable of the State monad. 

The difference to the functions `get` and `put` is that they are used outside of a State monad.

Two other helper functions that this module also contains are the `modify` and `gets` functions.
```haskell
modify :: (s -> s) -> State s ()
modify f = state $ \st -> ((), f st)

gets :: (s -> a) -> State s a
gets f = do
    st <- get
    return (f st)
```
They are also used inside a State monad.

The `modify` function takes in a function and updates the state monad such that is applies the function to the state variable.

The `gets` function takes in a function, applies it to the state variable and writes the result in the type `a` variable of the state monad.

### MonadState type class

The State monad and the helper functions presented in the previous two chapters can be also found in the **Control.Monad.Reader** module.

Same as for Reader and Writer monad this module is also part of the **mtl** package that extends the **transformers** package.

This module defines the `MonadReader` type class that contains the functions `get`, `put` and `state`.

They are equivalent to the functions describer in the previous chapter.

```haskell
class Monad m => MonadState s m | m -> s where
    get :: m s
    get = state (\s -> (s, s))

    put :: s -> m ()
    put s = state (\_ -> ((), s))

    state :: (s -> (a, s)) -> m a
    state f = do
      s <- get
      let ~(a, s') = f s
      put s'
      return a
    {-# MINIMAL state | get, put #-}
```

The minimal complete definition are the `get` and `put` functions or the `state` function. 

Same as for the `MonadReader` and `MonadWriter` type classes you need to use also here the *MultiParamTypeClasses* and *FunctionalDependencies* language pragma.

For our State monad we could write an instance of the `MonadState` type class as:
```haskell
instance MonadState s (State s) where
    put :: s -> State s ()
    put st = State $ \_ -> ((), st)

    get :: State s s
    get = State $ \st -> (st, st)
```

Also here you would need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

What all four language pragmas do was already explained in the Reader monad lesson. 

### Simple example

In practical Haskell code we do not define functions, types and type classes from the previous chapters on our own. 

We can import them from the **Control.Monad.State** module that uses Monad transformes. 

As said in the begining these functions and types work same as the one presented in previous chapters.

Let's write an example of a State monad that helps us compute the first n fibonacci numbers.

In [ ]:
import Control.Monad.State (MonadState(put, get), State, execState)

type FibNums = [Int]

printFibonacci :: IO ()
printFibonacci = do
    putStrLn "How many fibonacci numbers do you want to see:"
    n <- (read <$> getLine) :: IO Int
    if n < 1
    then do
        putStrLn "n has to be a positive number. Try again."
        printFibonacci
    else do
        putStrLn "Fibonacci numbers are:"
        if n == 1 || n == 2
        then print $ take n [0,1]
        else do
            let initialState = [n,0,1]
                finalState = execState calculateFib initialState
            print $ tail finalState

calculateFib :: State FibNums ()
calculateFib = do
    st <- get
    let newSt = st ++ [last st + last (init st)]
        finished = length (tail newSt) >= head newSt 
    put newSt
    if finished 
    then return ()
    else calculateFib

printFibonacci

**Code explanation**

First ask the user for a number and check if it is positive. In case that it is 1 or 2 no computation is needed and we print the list directly.

Else we run the **calculateFib** State monad that we provide with the initial state which is a list that contains the input number from the user and the first to fibonacci elements. 

The **calculateFib** State monad first gets the state, then it updates the state and checks if we already computed enough elements.

If not it recursively calls itself again and if yes it returns the state together with an empty tuple.

## State monad examples

Let's first look at the previous fibonacci example where we rewrite the `calculateFib` state monad such that we use the `modify` and `gets` functions.

In [ ]:
import Control.Monad.State (modify, gets)

calculateFib :: State FibNums ()
calculateFib = do
    let updateSt st = st ++ [last st + last (init st)]
        finished st = length (tail st) >= head st 

    modify updateSt
    finished <- gets finished
    if finished 
    then return ()
    else calculateFib

printFibonacci

If we use this version with the previous code it works the same. We see there is no need to use the `get` and `set` functions in this example.

Instead we define functions that take in a state and use them when we want to get and set the current state.

Next we write a game that takes in a string radomly composed of the characters `a`, `b` and `c`.

The game works like this: 
- we start with the intial state where the count is 0 and the game is OFF
- the caracter `c` switches the state of the game from OFF to ON or vice versa
- the caracter `a` adds +1 to the count if the game is ON
- the caracter `b` adds -1 to the count if the game is ON 

In the end the final count of the game is printed. We use the state monad to implement this game and also the `evalState` function.

In [ ]:
import Control.Monad.State (evalState, MonadState(put, get), State)

type GameValue = String
type GameState = (Bool, Int)

playGame :: String -> State GameState GameValue
playGame [] = do
    (_, score) <- get
    return $ "Final score is: " ++ show score
playGame (x : xs) = do
    (on, score) <- get
    case x of
        'a' | on -> put (on, score + 1)
        'b' | on -> put (on, score - 1)
        'c' -> put (not on, score)
        _ -> put (on, score)
    playGame xs

main :: IO ()
main = do
    let startState = (False, 0::Int)
    putStrLn $ evalState (playGame "acaaabcbbcbaa") startState

When using the pipe symbol `|` in a case expression it works as a logical AND and lets you add a condition to a case. 

In our last example we look at our initial password check example and how we can re-write it with the use of the State monad.

Our state variable will become the `CheckData` variable. And the return variable of the State monad will be a message of type `String`.

In [ ]:
import Control.Monad.State (MonadState(put, get), runState, State)
import Data.Char (toLower, toUpper)

type Message = String
type Password = String
type OperationsCount = Int
type CheckData = (OperationsCount, Password)

-- Removes duplicated characters
check1 :: State CheckData Message
check1 = do
    (count, password) <- get
    put (count + 1, filterPass password)
    if count > 20
    then do
        return "Hit recursion limit of 20 password checks."
    else check2
  where filterPass [] = []
        filterPass (x:xs) = x : filterPass (filter (filterFunc x) xs)
        filterFunc x = \char -> char `notElem` [toLower x,toUpper x]

-- Checks that the length of the password is not to small
check2 :: State CheckData Message
check2 = do
    (count, password) <- get
    if length password < 10
    then do
        put (count + 1, extendPassword password count)
        check1
    else do
        put (count + 1, password)
        check3
  where extendPassword password count = let shortage = 10 - length password
                                            passwordExtension = take shortage (drop count $ cycle allLetters)
                                        in password ++ passwordExtension

-- Checks if symbols are included
check3 :: State CheckData Message
check3 = do
    (count, password) <- get
    if noSymbols password
    then put (count + 1, extendPassword password count)
    else put (count + 1, password)
    check4 
  where noSymbols password = all (`elem` allLetters) password
        extendPassword password count = let passwordExtension = take 1 (drop count "@!#$%&()?=")
                                        in password ++ passwordExtension

-- Checks there are at least 1 upper and 1 smaller letters
check4 :: State CheckData Message
check4 = do
    (count, password) <- get
    if allLower password
    then do
        put (count + 1, upperExtendPassword password count)
        check1
    else if allUpper password
         then do 
            put (count + 1, lowerExtendPassword password count) 
            check1
        else do
            put (count + 1, password)
            return "Successfully performed all checks."
  where allLower password = password == map toLower password
        allUpper password = password == map toUpper password
        upperExtendPassword password count = password ++ take 1 (drop count $ cycle upperLetters)
        lowerExtendPassword password count = password ++ take 1 (drop count $ cycle lowerLetters)

lowerLetters :: String
lowerLetters = "abcdefghijklmnopqrstuvwxyz"

upperLetters :: String
upperLetters = "ABCDEFGHIJKLMNOPQRSTUVWXZY"

allLetters :: String
allLetters = lowerLetters ++ upperLetters

main :: IO ()
main = do
  putStrLn "Please input a password:"
  password <- getLine
  if null password
  then main
  else do
    let (msg,(count, checkedPassword)) = runState check1 (0,password)
    if password /= checkedPassword
    then do
        putStrLn msg
        putStrLn $ "Suggested improved passord: " ++ checkedPassword
    else putStrLn msg

main

We see that data does not need to be passed around since the state is propagated to the next monad that is called.

Also some of the logic for creating the user message that is printed at the end, is now moved into the State monads. 

## Recap

In this lesson we've discussed:

- the motivation for introducing the State monad type 

- the definition of the State monad type

- helper functions that work with the State monad

- examples that use the State monad type and its helper functions